In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from IPython.display import display


In [2]:
plt.style.use('fivethirtyeight')

In [3]:

datasets = ['nic.csv']
data = datasets[0]


In [4]:
df = pd.read_csv(data,index_col='Date')
df.head()

,S.N.,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
Date,,,,,,,
2019-01-01,1,30,7155,2707400,383,377,380
2019-01-02,2,43,19408,7370888,382,378,382
2019-01-03,3,44,8316,3179485,385,381,383
2019-01-06,4,48,9575,3631962,383,377,377
2019-01-07,5,45,17398,6606274,383,377,379


In [5]:
df.describe()

,S.N.,Total Transactions,Total Traded Shares,Total Traded Amount,Max. Price,Min. Price,Close Price
count,440.000000,440.000000,440.000000,4.400000e+02,440.000000,440.000000,440.000000
mean,220.500000,321.952273,80094.565909,4.772807e+07,508.490909,494.195455,500.438636
std,127.161315,393.480781,93826.650236,6.921361e+07,118.447801,110.842530,114.206187
min,1.000000,2.000000,60.000000,2.813000e+04,377.000000,368.000000,370.000000
25%,110.750000,88.000000,21959.500000,9.051618e+06,413.750000,405.750000,408.000000
50%,220.500000,171.000000,42403.000000,2.010977e+07,475.000000,465.000000,469.000000
75%,330.250000,388.000000,100493.000000,5.448131e+07,581.250000,570.000000,574.000000
max,440.000000,2871.000000,674434.000000,4.797399e+08,963.000000,912.000000,927.000000


In [6]:
def EMA(df, period = 12,column='Close Price'):
    return df[column].ewm(span=period,adjust=False).mean()


In [7]:
def MACD(df, low_period=12,high_period=26,signal_period=9,column='Close Price'):
    EMA_low = EMA(df,period=low_period,column=column)
    EMA_high = EMA(df,period=high_period,column=column)
    MACD_line= EMA_low - EMA_high
    signal_line = MACD_line.ewm(span=signal_period,adjust=False).mean()
    df['MACD'] = MACD_line
    df['Signal'] = signal_line
    return MACD_line,signal_line






In [21]:
def plot_MACD_signal(macd,signal,df=df,use_plotly=True,volume='Total Traded Shares',period_text='(12,26,9)'):
    if use_plotly:
        fig = go.Figure()
        fig = fig.add_trace(go.Scatter(x=df.index,y=macd,name='MACD',line=dict(color="green"), opacity=0.7))
        fig = fig.add_trace(go.Scatter(x=df.index,y=signal,name='Signal Line',line=dict(color="red"), opacity=0.7))
        fig.update_layout(
            title={
                'text': "MACD and Signal of ->"+ data+ period_text},
            legend_title="Legend",
            xaxis_title='Date',
            yaxis_title='MACD Value',
           
        )
        fig.add_trace(go.Scatter(x=df.index,y=np.zeros_like(signal),name='Zero Line',line=dict(color="black"), opacity=0.7))
        # fig.add_trace(go.Histogram(x=df.index,y=df[volume],name='Volume',opacity=0.7))

        

        fig.show()
        
    else:
        plt.figure(figsize=(10,5))
        plt.plot(df.index,macd,label='MACD',color='green',alpha=0.7)
        plt.plot(df.index,signal,color='red',label='Signal Line',alpha=0.7)
        plt.xticks(rotation=90,fontsize=6)
        plt.title("MACD and Signal of "+data)
        plt.legend(loc=0)
        plt.show()

In [16]:
def buy_sell(df,close='Close Price'):
    buy = []
    sell = []
    flag = -1

    for i in range(len(df)):
        if df['MACD'][i]>df['Signal'][i] and df['MACD'][i]<0 and flag != 1:
            #MACD crosses signal line from bottom to top below zero line
            flag = 1
            buy.append(df[close][i])
            sell.append(np.nan)
        elif df['MACD'][i] < df['Signal'][i]and df['MACD'][i]>0 and flag!= 0:
            #MACD crosses from top to below above zero line
            flag=0
            buy.append(np.nan)
            sell.append(df[close][i])
        else:
            buy.append(np.nan)
            sell.append(np.nan)
            
    df['Buy'] = buy
    df['Sell'] = sell

    return (buy,sell)


In [17]:
def plot_close_price(df,column='Close Price',plot_EMA=True,show_buy_sell=True,use_plotly=True,period_text='(12,26,9)'):
    if plot_EMA:
        ema10= EMA(df,period=10,column=column)
        ema100= EMA(df,period=100,column=column)
    if show_buy_sell:
        buy,sell = buy_sell(df)

    if use_plotly:

        fig = go.Figure()
        fig = fig.add_trace(go.Scatter(x=df.index,y=df[column],name='Close Price',opacity=0.7))
        if plot_EMA:
            fig = fig.add_trace(go.Scatter(x=df.index,y=ema10,name='EMA10',opacity=0.7))
            fig = fig.add_trace(go.Scatter(x=df.index,y=ema100,name='EMA100',opacity=0.7))
        if show_buy_sell:
            fig = fig.add_trace(go.Scatter(mode='markers',x=df.index,y=df['Buy'],name='Buy',marker=dict(
                color='green',
                size=10,
                symbol='triangle-up'
                
            ),
            opacity=0.7))

            fig = fig.add_trace(go.Scatter(mode='markers',x=df.index,y=df['Sell'],name='Sell',marker=dict(
                color='red',
                size=10,
                symbol='triangle-down'
                
            ),
             opacity=0.7))
        fig.update_layout(
            title={
                'text': "Close Price History ->"+ data+period_text},
            xaxis_title='Date',
            yaxis_title='Close Price'
        )
        fig.show()
    else:
        mpl_fig = plt.figure(figsize=(10,5))
        plt.plot(df.index,df[column],label='Close Price',color='blue',alpha=0.7)
        if plot_EMA:
            plt.plot(df.index,ema10,label='EMA10',color='green',alpha=0.7)
            plt.plot(df.index,ema100,label='EMA100',color='cyan',alpha=0.7)
        if show_buy_sell:
            plt.plot(df.index,df['Buy'],label='Buy',marker='^',alpha=1)
            plt.plot(df.index,df['Sell'],label='Sell',marker='^',alpha=1)
        plt.xticks(rotation=80,fontsize=5)
        plt.title("Close Price History "+ data)
        plt.legend(loc=0)
        plt.show()

In [11]:
macd,signal = MACD(df)

In [22]:
plot_close_price(df,use_plotly=True,plot_EMA=True)
plot_MACD_signal(macd,signal,df)

In [31]:

period = (12,30,8)

macd2,signal2 = MACD(df,*period)
plot_close_price(df,use_plotly=True,plot_EMA=True,period_text=str(period))
plot_MACD_signal(macd2,signal2,df,period_text=str(period))